# ⚡ Real-Time 5-Minute Trending Scout
**Topics:** Netflix · OpenAI  
**Window:** Last **5 minutes** only (Twitter v2 `start_time` filter)  
**Output:** Top 5 tweets ranked by engagement score per topic  

> ⚠️ **Twitter API note:** The free/Basic tier of Twitter v2 allows `recent search` (last 7 days). The 5-minute window is enforced by passing `start_time = now - 5min` as an ISO timestamp. If zero results come back, the cell automatically widens to **30 minutes** and flags it clearly.

In [ ]:
!pip install requests -q

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# ⚡ REAL-TIME 5-MINUTE TRENDING SCOUT
# Topics : Netflix | OpenAI
# Engine : Twitter v2 Recent Search (Yutori-style)
# Window : last 5 minutes → auto-widens to 15m → 30m if no results
# ─────────────────────────────────────────────────────────────────────────────

import json, time, requests
from getpass import getpass
from datetime import datetime, timezone, timedelta


# ─────────────────────────────────────────────────────────────────────────────
# ★ CONFIG
# ─────────────────────────────────────────────────────────────────────────────

TOPICS = {

    "Netflix": [
        # Breaking news / viral moments
        '(Netflix) (new OR just OR dropped OR announced OR trending OR viral OR breaking) -is:retweet lang:en',
        # Show / movie release chatter
        '(Netflix) (series OR show OR film OR movie OR season OR episode OR cancel OR renew) -is:retweet lang:en',
        # Price / service news
        '(Netflix) (price OR subscription OR password OR login OR outage OR down) -is:retweet lang:en',
    ],

    "OpenAI": [
        # Announcements / releases
        '(OpenAI) (release OR launch OR dropped OR announced OR just OR new) -is:retweet lang:en',
        # Model / product chatter
        '(OpenAI) (GPT OR ChatGPT OR model OR agent OR API OR Sora OR o3 OR o4) -is:retweet lang:en',
        # Controversy / breaking
        '(OpenAI) (viral OR breaking OR controversy OR fired OR news OR drama) -is:retweet lang:en',
    ],
}

# Time window cascade (minutes) — tries each in order until results found
WINDOW_CASCADE_MINUTES = [5, 15, 30]

TOP_N          = 5     # tweets to show per topic
PULL_PER_QUERY = 100   # max per API call (Twitter max = 100)


# ─────────────────────────────────────────────────────────────────────────────
# TWITTER v2 SEARCH
# ─────────────────────────────────────────────────────────────────────────────

def twitter_search(query: str, bearer: str, start_time: str, max_results: int = 100) -> list:
    """
    Hit Twitter v2 /tweets/search/recent with a start_time filter.
    Returns raw list of enriched tweet dicts, sorted by engagement.
    """
    url     = "https://api.twitter.com/2/tweets/search/recent"
    headers = {"Authorization": f"Bearer {bearer}"}
    params  = {
        "query":        query,
        "start_time":   start_time,          # ← this pins the 5-min window
        "max_results":  min(max(max_results, 10), 100),
        "tweet.fields": "public_metrics,created_at,author_id,entities,possibly_sensitive",
        "expansions":   "author_id",
        "user.fields":  "name,username,verified,public_metrics",
    }

    try:
        r = requests.get(url, headers=headers, params=params, timeout=15)
        r.raise_for_status()
    except requests.exceptions.HTTPError as e:
        print(f"   ⚠️  HTTP {r.status_code}: {r.text[:200]}")
        return []
    except Exception as e:
        print(f"   ⚠️  Request error: {e}")
        return []

    data   = r.json()
    tweets = data.get("data", [])
    users  = {u["id"]: u for u in data.get("includes", {}).get("users", [])}

    results = []
    for t in tweets:
        m      = t.get("public_metrics", {})
        author = users.get(t.get("author_id", ""), {})
        uname  = author.get("username", "unknown")

        # Engagement score: weighted sum
        likes       = m.get("like_count", 0)
        retweets    = m.get("retweet_count", 0)
        replies     = m.get("reply_count", 0)
        quotes      = m.get("quote_count", 0)
        impressions = m.get("impression_count", 0)

        engagement = (
            likes       * 1
            + retweets  * 4    # retweets = strongest virality signal
            + replies   * 2
            + quotes    * 3
            + impressions // 1000   # impressions normalized
        )

        results.append({
            "tweet_id":    t["id"],
            "url":         f"https://twitter.com/{uname}/status/{t['id']}",
            "text":        t["text"],
            "author":      author.get("name", ""),
            "username":    uname,
            "verified":    author.get("verified", False),
            "followers":   author.get("public_metrics", {}).get("followers_count", 0),
            "likes":       likes,
            "retweets":    retweets,
            "replies":     replies,
            "quotes":      quotes,
            "impressions": impressions,
            "engagement":  engagement,
            "created_at":  t.get("created_at", ""),
            "query":       query,
        })

    results.sort(key=lambda x: x["engagement"], reverse=True)
    return results


def scout_topic(topic: str, queries: list, bearer: str,
                window_cascade=WINDOW_CASCADE_MINUTES, pull=PULL_PER_QUERY):
    """
    Run all queries for a topic. Auto-widen time window if no results found.
    Returns (tweets_list, actual_window_minutes_used).
    """
    for minutes in window_cascade:
        start_time = (
            datetime.now(timezone.utc) - timedelta(minutes=minutes)
        ).strftime("%Y-%m-%dT%H:%M:%SZ")

        print(f"   ⏱️  Window: last {minutes} min  (start_time={start_time})")

        seen, all_tweets = set(), []
        for q in queries:
            tweets = twitter_search(q, bearer, start_time=start_time, max_results=pull)
            for t in tweets:
                if t["tweet_id"] not in seen:
                    seen.add(t["tweet_id"])
                    all_tweets.append(t)
            time.sleep(0.5)

        all_tweets.sort(key=lambda x: x["engagement"], reverse=True)

        if all_tweets:
            print(f"   ✅ {len(all_tweets)} unique tweets found in last {minutes} min")
            return all_tweets, minutes
        else:
            print(f"   ⚠️  No results in last {minutes} min — widening window...")

    print(f"   ❌ No results found even at {window_cascade[-1]} min window.")
    return [], window_cascade[-1]


# ─────────────────────────────────────────────────────────────────────────────
# DISPLAY
# ─────────────────────────────────────────────────────────────────────────────

def print_top_tweets(topic: str, tweets: list, window_min: int, limit: int = TOP_N):
    now_str = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S UTC")
    label   = "⚡ LIVE" if window_min <= 5 else f"🕐 Last {window_min} min"

    print(f"\n{'='*72}")
    print(f"  🏆 TOP {limit} TWEETS — {topic.upper()}")
    print(f"  {label}  |  Scraped at: {now_str}")
    print(f"{'='*72}")

    if not tweets:
        print(f"  😶 No tweets found for '{topic}' in the last {window_min} minutes.")
        return

    for i, t in enumerate(tweets[:limit]):
        ts       = t['created_at'].replace('T',' ').replace('Z',' UTC') if t['created_at'] else 'N/A'
        verified = "✅" if t['verified'] else ""
        print(f"""
  #{i+1:02d}  🐦 @{t['username']} {verified}  ({t['followers']:,} followers)
       📅 {ts}
       ❤️  {t['likes']:,}   🔁 {t['retweets']:,}   💬 {t['replies']:,}   🔖 {t['quotes']:,}   👁 {t['impressions']:,}
       📊 Engagement score: {t['engagement']:,}
       🔗 {t['url']}
       💬 {t['text'][:300]}
  {'─'*70}""")


# ─────────────────────────────────────────────────────────────────────────────
# MAIN
# ─────────────────────────────────────────────────────────────────────────────

print("🔑 Enter your Twitter Bearer Token")
print("   (Get one free at developer.twitter.com → Projects → Bearer Token)")
BEARER = getpass("Twitter Bearer Token: ")

all_results = {}

for topic, queries in TOPICS.items():
    print(f"\n{'='*72}")
    print(f"⚡ SCOUTING: {topic.upper()}")
    print(f"{'='*72}")

    tweets, window_used = scout_topic(
        topic   = topic,
        queries = queries,
        bearer  = BEARER,
    )

    all_results[topic] = {
        "window_minutes": window_used,
        "total_found":    len(tweets),
        "top_5":          tweets[:TOP_N],
    }

    print_top_tweets(topic, tweets, window_used, limit=TOP_N)

# Save combined output
with open("realtime_trending_top5.json", "w") as f:
    json.dump(all_results, f, indent=2, default=str)

print("\n✅ Saved: realtime_trending_top5.json")
print("🏁 Done.")

🔑 Enter your Twitter Bearer Token
   (Get one free at developer.twitter.com → Projects → Bearer Token)
Twitter Bearer Token: ··········

⚡ SCOUTING: NETFLIX
   ⏱️  Window: last 5 min  (start_time=2026-02-27T23:29:11Z)
   ✅ 26 unique tweets found in last 5 min

  🏆 TOP 5 TWEETS — NETFLIX
  ⚡ LIVE  |  Scraped at: 2026-02-27 23:34:13 UTC

  #01  🐦 @Howodd69   (977 followers)
       📅 2026-02-27 23:33:55.000 UTC
       ❤️  0   🔁 0   💬 1   🔖 0   👁 1
       📊 Engagement score: 2
       🔗 https://twitter.com/Howodd69/status/2027527646469624068
       💬 1- The future of John Oliver's 'Last Week Tonight' will be a stress test for Paramount. Netflix dropping out of the Warner Bros. Discovery deal has raised a massive question in the world of late night: Will John Oliver stay around long enough to meet his new business daddy? https://t.co/fZs1a9lwzP
  ──────────────────────────────────────────────────────────────────────

  #02  🐦 @CuriousMrFox101   (1,696 followers)
       📅 2026-02-27 23:31:23.

---
## 🔁 Re-run every 5 minutes (optional polling loop)
Uncomment the cell below to continuously poll every 5 minutes.

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# OPTIONAL: Continuous 5-minute polling loop
# Uncomment and run this cell to keep refreshing automatically.
# Press the ■ STOP button in Colab to end the loop.
# ─────────────────────────────────────────────────────────────────────────────

# POLL_INTERVAL_SECONDS = 300   # 5 minutes
# POLL_ROUNDS = 12              # run for 1 hour (12 × 5min)

# for round_num in range(1, POLL_ROUNDS + 1):
#     print(f"\n\n{'#'*72}")
#     print(f"# POLL ROUND {round_num}/{POLL_ROUNDS}  —  {datetime.now(timezone.utc).strftime('%H:%M:%S UTC')}")
#     print(f"{'#'*72}")
#
#     for topic, queries in TOPICS.items():
#         tweets, window_used = scout_topic(topic=topic, queries=queries, bearer=BEARER)
#         print_top_tweets(topic, tweets, window_used, limit=TOP_N)
#
#     if round_num < POLL_ROUNDS:
#         print(f"\n⏳ Next poll in {POLL_INTERVAL_SECONDS // 60} min...")
#         time.sleep(POLL_INTERVAL_SECONDS)